In [1]:
#JV

I would develop the code for the assignment in this notebook as it is easy to quickly test (and even unit testing).

When a module/part is bug free I would add it to the .py file later.



In [2]:
import os
import shutil
import re

import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
def make_dir(dir,returnIfDirAlreadyExists=False):
    """
    Function to create a directory, if it doesn't exist
    """
    try:
        os.mkdir(dir)
    except Exception as e:
        if "File exists" in str(e):
            if returnIfDirAlreadyExists:
                return True
            pass
        else:
            print(e)

In [4]:
seed = 76 #setting this as seed wherever randomness comes

In [5]:
## Train data downloaded from the given source (https://storage.googleapis.com/wandb_datasets/nature_12K.zip)

"""
Now, the goal is to split 20% of train data, in "train" folder to get validation data.
"""



data_base_dir = 'inaturalist_12K/'

def train_validation_split(base_dir,seed = 76):
    """
    Function to split 20% of the train data into validation data, Uniformly At Random (UAR). Import os and shutil before using this method.

    Note  : Instead of taking 20% of samples randomly out of the entire train data; 20% of train data of each class is taken (UAR), 
    so that for training there is a balance between the number of samples per class.

    Params:

        base_dir : The path to the directory in which the "train/" and "test/" directories are present after unzipping. It is assumed that the given dir path string has a "/ at the end.

        seed : The seed use in the random number generator, default : 76.

    Returns :

        None.
    """

    base_data_dir = base_dir
    train_base_dir = base_data_dir+'train/'
    train_data_class_dirs = os.listdir(train_base_dir)
    
    ## remove dirs starting with "." from the list
    train_data_class_dirs = [i for i in train_data_class_dirs if i[0] != "." ]

    ## Test data is called as val, which is confusing, hence renaming it to test
    os.rename(data_base_dir+"val/",base_data_dir+"test/")
    
    
    ## validation dir
    val_base_dir = base_data_dir+'validation/'
    make_dir(val_base_dir)
    
    ## Iterate over each class and
    ## take 20% data of each class at random as validation data
    
    random_num_generator = np.random.RandomState(seed)
    
    for class_label in train_data_class_dirs:
    
        current_class_train_filenames = os.listdir(train_base_dir+class_label+"/")
    
        num_of_files = len(current_class_train_filenames)
        
        validation_indices = random_num_generator.choice(num_of_files,int(0.2*num_of_files),replace=False)
        train_indices = np.array(list(set(np.arange(num_of_files)).difference(set(validation_indices))))
    
        ##create class dir validation dir
        cur_validation_dir = val_base_dir + class_label +"/"
        make_dir(cur_validation_dir)
        
        for i in validation_indices:
            shutil.move(train_base_dir+class_label+"/"+current_class_train_filenames[i],cur_validation_dir+current_class_train_filenames[i])
        
        print(f"Validation Split for {class_label} is Done!")



In [6]:
"""
Careful perform this train-validation split only once in the entire lifetime, that too on the unzipped dataset.

"""

base_data_dir = "inaturalist_12K/"

#train_validation_split(base_data_dir)

In [7]:
""" Create loader for train, test and validation data """

## Train data
train_path = base_data_dir+"train/"
train_dataset = torchvision.datasets.ImageFolder(root=train_path,transform=torchvision.transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16,shuffle=True,num_workers=0)

## Validation data
val_path = base_data_dir+"validation/"
val_dataset = torchvision.datasets.ImageFolder(root=val_path,transform=torchvision.transforms.ToTensor())
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=16,shuffle=True,num_workers=0)

## Test data
test_path = base_data_dir+"test/"
test_dataset = torchvision.datasets.ImageFolder(root=test_path,transform=torchvision.transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=16,shuffle=True,num_workers=0)

In [ ]:
class CNN(nn.Module):
    """
    A class (that inherits nn.Module), to create the CNN architecture as required and to define the forward pass.
    """

    def __init__(self,input_depth,num_output_neurons, activation, convolution_layer_specifications, hidden_layer_specifications, output_activation):

        """
        Default Constructor.

        Params:

            input_depth : Depth of the input image (number of channels).
            
            num_output_neurons: Number of neurons in the output layer.

            activation : A torch nn method to be used as activation function.
            
            convolution_layer_specifications: A list of lists. There exists one list per conv. layer contaning, containing number of filters, filter sizes, paddings.
            
            hidden_layer_specifications: A list of ints. Number of elements correspond to number of hidden layers and each value gives the number of neurons in the corresponding hidden layer.
            
            output_activation: The torch nn activation method to be used for the output layer.


        Returns:
        
            None.
        """

        super(CNN, self).__init__()

        self.output_activation = output_activation

        self.convolutional_layers = nn.ModuleList() ## Create a module list to organize the convolutional layers
        input_channels = input_depth
        
        ## iterate over the convolution_layer_specifications and create the convolutional layers accordingly
        for number_of_filters, filter_size, stride, padding, max_pool_dim in convolution_layer_specifications:

            ## Assuming filter is a square matrix, so filter_size is int.
            convolutional_layers.append(nn.Conv2d(input_channels, number_of_filters, filter_size, stride, padding))
            convolutional_layers.append(activation)  # Add activation after each convolutional layer
            convolutional_layers.append(nn.MaxPool2d(max_pool_dim))
            
            input_channels = number_of_filters  # Update input depth for next layer


        ## iterate over the hidden_layer_specifications and create CNN layers accordingly.
        self.hidden_layers = nn.ModuleList()
        
        fan_in = """What should come here?""" ## interface betweeon maxpooling and dense layer
        for hidden_size in hidden_layer_specifications
            hidden_layers.append(nn.Linear(fan_in, hidden_size))
            hidden_layers.append(nn.ReLU())  # Add ReLU activation after each dense layer
            fan_in = hidden_size  # Update number of input features for next layer


        self.output_layer = nn.Linear(hidden_layer_specifications[-1], num_output_neurons)


        def forward(self,x):

            ## pass through convolution, activation, pooling layer set
            for layer in self.convolutional_layers:
                x = layer(x)
            
            x = torch.flatten(x, 1)


            ## pass through hidden layers
            for layer in self.dense_layers:
            x = layer(x)

            ## compute and activate the output
            output = self.output_activation(self.output_layer(x))
            
            return output
            

### References:

1. https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
2. https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#:~:text=PyTorch%20provides%20two%20data%20primitives,easy%20access%20to%20the%20samples.
3. https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
4. https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html